## Import Libraries

In [1]:
!pip install wandb --quiet
!pip install torchsummaryX -q
!pip install datasets
!pip install zstandard
!pip install tiktoken
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.1 MB/s eta 0:00:00


In [2]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
import torchaudio.transforms as tat
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime
import zstandard
import datasets
import tiktoken
import random
import wandb
import math

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [3]:
### If you are using colab, you can import google drive to save model checkpoints in a folder.
### This is used when connecting to GCE VMs, but the user still wants to connect to Google Drive
import os.path as path
if not path.exists("/content/drive"):
  !sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !sudo apt-get update -qq 2>&1 > /dev/null
  !sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
  !google-drive-ocamlfuse

  !sudo apt-get install -qq w3m # to act as web browser
  !xdg-settings set default-web-browser w3m.desktop # to set default browser
  %cd /content
  !mkdir drive
  %cd drive
  !mkdir MyDrive
  %cd ..
  %cd ..
  !google-drive-ocamlfuse /content/drive/MyDrive



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

# Downloading Dataset (Stanford QA)

In [83]:
from datasets import load_dataset
import datasets
dataset = load_dataset('squad')

In [84]:
config = {
    'epochs'        : 5,
    'batch_size'    : 32,
    'init_lr'       : 3e-4,
    'block_size'    : 256,
    'dropout'       : 0.1,
    'vocab_size'    : 50257,
    'bias'          : True,
    'n_layer'       : 12,
    'n_head'        : 10,
    'n_embd'        : 250,
    'end_token'     : 50256,
    'answer_length' : 15
}

In [85]:
test_valid = dataset['validation'].train_test_split(test_size=0.5)
dataset = datasets.DatasetDict({
    'train': dataset['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [86]:
def pad_with_eos(text, length=config['answer_length']):
  if length > len(text):
    pad_len = length - len(text)
    text += [config['end_token']] * pad_len
    return text
  else:
    return text[:length]

In [127]:
# Dataset class to load train and validation data

class StanfordQADataset(torch.utils.data.Dataset):

    def __init__(self, prefix):

        prev_data = dataset[prefix]
        enc = tiktoken.get_encoding("gpt2")

        # Only get the first 256 tokens of the question and the context (to account for block size)
        self.inputs = [enc.encode_ordinary("Answer the following question given the context. Question: " + prev_data[i]["question"] +
                        " Context: " + prev_data[i]["context"])[:min(256, len(enc.encode_ordinary(prev_data[i]["question"] + prev_data[i]["context"]))+12)]
            if (prev_data[i]["question"] != None and prev_data[i]["context"] != None) else [] for i in range(len(prev_data))]

        self.targets = [enc.encode_ordinary(prev_data[i]["answers"]["text"][0]) for i in range(len(prev_data))]

        assert len(self.inputs) == len(self.targets)
        self.length = len(self.inputs)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        return torch.tensor(self.inputs[ind]), torch.tensor(self.targets[ind])

    def collate_fn(self,batch):
        # batch of input mfcc coefficients
        batch_question = [batch[i][0] for i in range(len(batch))]
        # batch of output phonemes
        batch_answer = [batch[i][1] for i in range(len(batch))]

        # Pad the sequences so that they are all the same lengths in batch
        batch_question_pad = pad_sequence(batch_question, batch_first=True, padding_value=config['end_token'])
        B, T = batch_question_pad.shape
        batch_question_pad = torch.cat([batch_question_pad, torch.full((B,256-T), fill_value=config['end_token'], dtype=torch.int32)], dim=1)
        lengths_question = [batch_question[question].shape[0] for question in range(len(batch_question))]

        batch_answer_pad = pad_sequence(batch_answer, batch_first=True, padding_value=config['end_token'])
        B, T = batch_answer_pad.shape
        if (T >= 15):
          batch_answer_pad = batch_answer_pad[:, :config['answer_length']]
        else:
          batch_answer_pad = torch.cat([batch_answer_pad, torch.full((B,config['answer_length']-T), fill_value=config['end_token'], dtype=torch.int32)], dim=1)
        lengths_answer = [batch_answer[answer].shape[0] for answer in range(len(batch_answer))]

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_question_pad, batch_answer_pad, torch.tensor(lengths_question), torch.tensor(lengths_answer)

In [128]:
# Dataset class to load train and validation data

class StanfordQATestDataset(torch.utils.data.Dataset):

    def __init__(self, prefix):

        prev_data = dataset[prefix]
        enc = tiktoken.get_encoding("gpt2")

        # Only get the first 256 tokens of the question and the context (to account for block size)
        self.inputs = [enc.encode_ordinary("Answer the following question given the context. Question: " + prev_data[i]["question"] +
                        " Context: " + prev_data[i]["context"] + " Answer: ")[:min(256, len(enc.encode_ordinary(prev_data[i]["question"] + prev_data[i]["context"]))+14)]
            if (prev_data[i]["question"] != None and prev_data[i]["context"] != None) else [] for i in range(len(prev_data))]

        self.length = len(self.inputs)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        return torch.tensor(self.inputs[ind])

    def collate_fn(self,batch):
        # Pad the sequences so that they are all the same lengths in batch
        batch_question_pad = pad_sequence(batch, batch_first=True, padding_value=config['end_token'])
        B, T = batch_question_pad.shape
        batch_question_pad = torch.cat([batch_question_pad, torch.full((B,256-T),fill_value=config['end_token'],dtype=torch.int32)], dim=1)
        lengths_question = [batch[question].shape[0] for question in range(len(batch))]

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_question_pad, torch.tensor(lengths_question)

In [129]:
train_data = StanfordQADataset(prefix="train")
val_data = StanfordQADataset(prefix="valid")
test_data = StanfordQADataset(prefix="test")

In [130]:
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 1,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    collate_fn  = train_data.collate_fn,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 1,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    collate_fn  = val_data.collate_fn,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 1,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    collate_fn  = test_data.collate_fn,
    shuffle     = False
)

In [131]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    x, y, lx, ly = data
    print(x.shape, y.shape)
    print(x, y)
    break

for i, data in enumerate(test_loader):
    x, y, lx, ly = data
    print(x.shape, y.shape)
    print(x, y)
    break

torch.Size([32, 256]) torch.Size([32, 15])
tensor([[33706,   262,  1708,  ...,    11,   673,  9301],
        [33706,   262,  1708,  ..., 50256, 50256, 50256],
        [33706,   262,  1708,  ..., 50256, 50256, 50256],
        ...,
        [33706,   262,  1708,  ...,   423,  2098,   326],
        [33706,   262,  1708,  ..., 50256, 50256, 50256],
        [33706,   262,  1708,  ..., 50256, 50256, 50256]]) tensor([[  298,  7537,    72, 16607, 15846,   287,  1123, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [ 1731,  2075, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [ 1169,  2039, 22873,  2634, 11979, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [ 1065,  1120,  9088, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [ 3791,  4479, 21345, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 5025

In [132]:
gc.collect()
torch.cuda.empty_cache()

## Blocks

In [133]:
# Layer normalization for regularizing the model
class LayerNorm(nn.Module):
  def __init__(self, ndim, bias):
    super().__init__()
    self.weight = torch.nn.Parameter(torch.ones(ndim))
    self.bias = torch.nn.Parameter(torch.zeros(ndim)) if bias else None

  def forward(self, input):
    return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

In [134]:
class CausalSelfAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.c_attn = nn.Linear(config['n_embd'], 3 * config['n_embd'], bias=config['bias'])
    self.c_proj = nn.Linear(config['n_embd'], config['n_embd'], bias=config['bias'])

    self.attn_dropout = nn.Dropout(config['dropout'])
    self.resid_dropout = nn.Dropout(config['dropout'])

  def forward(self, x):
    B, T, C = x.size()

    q, k, v  = self.c_attn(x).split(config['n_embd'], dim=2)
    k = k.view(B, T, config['n_head'], C // config['n_head']).transpose(1, 2)
    q = q.view(B, T, config['n_head'], C // config['n_head']).transpose(1, 2)
    v = v.view(B, T, config['n_head'], C // config['n_head']).transpose(1, 2)

    y = torch.nn.functional.scaled_dot_product_attention(q, k, v,
                                                         attn_mask=None,
                                                         dropout_p=config['dropout']if self.training else 0,
                                                         is_causal=True)

    y = y.transpose(1, 2).contiguous().view(B, T, C)

    y = self.resid_dropout(self.c_proj(y))
    return y

In [135]:
class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config['n_embd'], 4 * config['n_embd'], bias=config['bias'])
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4 * config['n_embd'], config['n_embd'], bias=config['bias'])
        self.dropout = nn.Dropout(config['dropout'])

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

In [136]:
class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config['n_embd'], bias=config['bias'])
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config['n_embd'], bias=config['bias'])
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

## Model

In [137]:
class GPT(nn.Module):
  def __init__(self, config):
    super().__init__()

    self.config = config

    self.transformer = nn.ModuleDict(dict(
        wte = nn.Embedding(config['vocab_size'], config['n_embd']),
        wpe = nn.Embedding(config['block_size'], config['n_embd']),
        drop = nn.Dropout(config['dropout']),
        h = nn.ModuleList([Block(config) for _ in range(config['n_layer'])]),
        ln_f = LayerNorm(config['n_embd'], bias=config['bias']),
    ))

    self.lm_head = nn.Linear(config['n_embd'], config['vocab_size'], bias=False)

    self.transformer.wte.weight = self.lm_head.weight
    self.apply(self._init_weights)

    for pn, p in self.named_parameters():
      if pn.endswith('c_proj.weight'):
        torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config['n_layer']))


  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        if module.bias is not None:
            torch.nn.init.zeros_(module.bias)
    elif isinstance(module, nn.Embedding):
        torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

  def forward(self, idx, targets=None):
      device = idx.device
      b, t = idx.size()
      pos = torch.arange(0, t, dtype=torch.long, device=device)

      # forward the GPT model itself
      tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
      pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
      x = self.transformer.drop(tok_emb + pos_emb)
      for block in self.transformer.h:
          x = block(x)
      x = self.transformer.ln_f(x)

      logits = self.lm_head(x)

      return logits

  def generate(self, idx, max_new_tokens, temperature=1.0):
    for _ in range(max_new_tokens):
        idx_cond = idx if idx.size(1) <= self.config['block_size'] else idx[:, -self.config['block_size']:]
        logits, _ = self(idx_cond)
        logits = logits[:, -1, :] / temperature
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

In [138]:
model = GPT(config).to(device)
summary(model, x.to(device))

                                                 Kernel Shape  \
Layer                                                           
0_transformer.Embedding_wte                      [250, 50257]   
1_transformer.Embedding_wpe                        [250, 256]   
2_transformer.Dropout_drop                                  -   
3_transformer.h.0.LayerNorm_ln_1                        [250]   
4_transformer.h.0.attn.Linear_c_attn               [250, 750]   
5_transformer.h.0.attn.Linear_c_proj               [250, 250]   
6_transformer.h.0.attn.Dropout_resid_dropout                -   
7_transformer.h.0.LayerNorm_ln_2                        [250]   
8_transformer.h.0.mlp.Linear_c_fc                 [250, 1000]   
9_transformer.h.0.mlp.GELU_gelu                             -   
10_transformer.h.0.mlp.Linear_c_proj              [1000, 250]   
11_transformer.h.0.mlp.Dropout_dropout                      -   
12_transformer.h.1.LayerNorm_ln_1                       [250]   
13_transformer.h.1.attn.L

<All keys matched successfully>

In [139]:
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(), lr= config['init_lr']) #Defining Optimizer
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor=0.75)
scaler = torch.cuda.amp.GradScaler()

In [140]:
def generate_text_logits(model, seq, max_new_tokens=config['answer_length']):
  text_logits = None

  for i in range(max_new_tokens):
      seq_cond = seq if seq.size(1) <= config['block_size'] else seq[:, -config['block_size']:]
      logits = model(seq_cond)

      if i == max_new_tokens-1:
        text_logits = logits[:, -max_new_tokens:, :] # batch size, max_new_tokens, vocab_size

      logits = logits[:, -1, :]
      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      seq = torch.cat((seq, idx_next), dim=1)

  return text_logits, seq[:, -max_new_tokens:]

In [141]:
def train(model, dataloader, optimizer):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    scaler = torch.cuda.amp.GradScaler()
    counter = 0

    for i, (inputs, targets, lx, ly) in enumerate(dataloader):

        counter += 1

        ### Initialize Gradients
        optimizer.zero_grad()

        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
          ### Move Data to Device (Ideally GPU)
          inputs      = inputs.to(device)
          targets    = targets.to(device)

          ### Forward Propagation
          logits, _ = generate_text_logits(model, inputs)

          B, T, C = logits.shape

          logits = logits.reshape(B*T, -1)
          targets = targets.reshape(-1)

          loss =  criterion(logits, targets)

        ### Backward Propagation
        scaler.scale(loss).backward()
        # loss.backward()

        ### Gradient Descent
        scaler.step(optimizer)

        scaler.update()

        tloss   += loss.item()

        batch_bar.set_postfix(loss="{:.07f}".format(float(tloss / (i + 1))))
        batch_bar.update()

        if (counter % 100 == 0):
          wandb.log({'train_loss': (tloss/counter), 'lr': curr_lr})

        ### Release memory
        del inputs, targets, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)

    return tloss

In [142]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc, vdist = 0, 0, 0 # Monitoring loss, accuracy, and distance
    batch_bar   = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    counter = 0

    for i, (inputs, targets, lx, ly) in enumerate(dataloader):

        counter += 1

        ### Move data to device (ideally GPU)
        inputs      = inputs.to(device)
        targets    = targets.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            logits, _ = generate_text_logits(model, inputs)

            B, T, C = logits.shape

            logits = logits.reshape(B*T, -1)
            targets = targets.reshape(-1)

            loss =  criterion(logits, targets)

        vloss   += loss.item()
        batch_bar.set_postfix(loss="{:.07f}".format(float(vloss / (i + 1))))
        batch_bar.update()

        if (counter % 20 == 0):
          wandb.log({'val_loss': (vloss/counter), 'dist/lr': curr_lr})

        ### Release memory
        del inputs, targets, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vdist   /= len(val_loader)

    return vloss

In [143]:
wandb.login(key="9312acc23a6389a925ba54b1bdf81ff99fe4d2e4") # API key for the project

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [144]:
# Create your wandb run
run = wandb.init(
    name    = "use-generation-function", ### Wandb last name initializer
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    project = "hw5-finetune", ### Project should be created in WandB
    config  = config ### Wandb Config for your run
)

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()

for epoch in range(1, config['epochs']):

    print("\nEpoch {}/{}".format(epoch, config['epochs']))

    curr_lr      = float(optimizer.param_groups[0]['lr'])
    train_loss   = train(model, train_loader, optimizer)
    val_loss     = eval(model, val_loader)

    print("\tTrain Loss {:.07f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Loss {:.07f}".format(val_loss))
    wandb.log({'train_loss': train_loss, 'valid_loss': val_loss, 'lr': curr_lr})

    scheduler.step(val_loss)


Epoch 1/5


	Train Loss 2.2416213	 Learning Rate 0.0003000
	Val Loss 2.4287194

Epoch 2/5


Train: 100%|█████████▉| 2731/2738 [1:19:42<00:12,  1.75s/it, loss=2.0429901]

In [27]:
def generate(idx, max_new_tokens, temperature=1.0, top_k=None):
    """
    Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
    the sequence max_new_tokens times, feeding the predictions back into the model each time.
    Most likely you'll want to make sure to be in model.eval() mode of operation for this.
    """
    for _ in range(max_new_tokens):
        # if the sequence context is growing too long we must crop it at block_size
        idx_cond = idx if idx.size(1) <= config['block_size'] else idx[:, -config['block_size']:]
        # forward the model to get the logits for the index in the sequence
        logits = model(idx_cond)
        # pluck the logits at the final step and scale by desired temperature
        logits = logits[:, -1, :] / temperature
        # optionally crop the logits to only the top k options
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = -float('Inf')
        # apply softmax to convert logits to (normalized) probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1)
        # append sampled index to the running sequence and continue
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

In [61]:
def calculate_rouge(output, label, ly): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    enc = tiktoken.get_encoding("gpt2")
    rouge = Rouge()

    for i in range(batch_size):
        pred_list = output[i].tolist()
        if (config['end_token'] in pred_list):
          temp = pred_list.index(config['end_token'])
          pred_list = pred_list[:temp]
        pred_string = enc.decode(pred_list) if pred_list != [] else "No answer."
        label_string = enc.decode(label[i][:ly[i]].tolist())
        dist += rouge.get_scores(pred_string, label_string)[0]["rouge-l"]["f"]

        if (i == 0):
          print(f"PREDICTION STRING: {pred_string}\n")
          print(f"LABEL STRING: {label_string}\n")

    dist /= batch_size
    return dist

In [62]:
total_loss = 0
for data in tqdm(test_loader):
    x, y, lx, ly = data
    x = x.to(device)

    # print(x.shape, y.shape)

    with torch.no_grad():
      raw_predictions = generate(x, 15)

    print(raw_predictions.shape)
    raw_predictions = raw_predictions[:, -15:]

    print(raw_predictions.shape)

    loss = calculate_rouge(raw_predictions, y, ly)
    print(loss)

    total_loss += loss

total_loss /= len(test_loader)
print(total_loss)

  1%|          | 1/166 [00:02<07:10,  2.61s/it]

torch.Size([32, 271])
torch.Size([32, 15])
PREDICTION STRING: 1 HOW TOKEY 




The Straw coupon bar

LABEL STRING: a rendezvous —let alone a docking

0.00416666656666667


  1%|          | 2/166 [00:05<06:59,  2.56s/it]

torch.Size([32, 271])
torch.Size([32, 15])
PREDICTION STRING: The government agency is issuing a witness report from NYC's Super Bowl photo.

LABEL STRING: higher economic inequality

0.01156938774866147


  2%|▏         | 3/166 [00:07<06:53,  2.53s/it]

...

torch.Size([32, 271])
torch.Size([32, 15])
PREDICTION STRING: 




The Cocquerade industry '? in block builder


LABEL STRING: Warsaw

0.012192234384552665


 99%|█████████▉| 165/166 [06:56<00:02,  2.51s/it]

torch.Size([32, 271])
torch.Size([32, 15])
PREDICTION STRING:  learning himself to keep the Hanss News) at the Henryapolis Monastery

LABEL STRING: Paul Marin de la Malgue

0.011346725812163422


100%|██████████| 166/166 [06:56<00:00,  2.51s/it]

torch.Size([5, 271])
torch.Size([5, 15])
PREDICTION STRING: Indiana



AIPDH President Donald Trump has contacted George Tech

LABEL STRING: ten seasons

0.024999999140625034
0.007730384763542612


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/DL/hw5-finetune/initial-try/model_checkpoint_epoch_2.pth'))

"Answer the following question given the context. Question: What makes day length constant on Earth? Context: The concept of inertia can be further generalized to explain the tendency of objects to continue in many different forms of constant motion, even those that are not strictly constant velocity. The rotational inertia of planet Earth is what fixes the constancy of the length of a day and the length of a year. Albert Einstein extended the principle of inertia further when he explained that reference frames subject to constant acceleration, such as those free-falling toward a gravitating object, were physically equivalent to inertial reference frames. This is why, for example, astronauts experience weightlessness when in free-fall orbit around the Earth, and why Newton's Laws of Motion are more easily discernible in such environments. If an astronaut places an object with mass in mid-air next to himself, it will remain stationary with respect to the astronaut due to its inertia. Th

In [ ]:
enc.decode(list(output[0]))

'Anxiety disorders are the most prevalent of all brain disorders, and yet there’s been little progress in drug treatments for anxiety in more than 50 years. Now, researchers reporting in the Cell Press journal Current Biology on March 24 suggest that studies in flies might help to elucidate fundamental mechanisms underlying anxiety and point the way to new anti-anxiety treatments. “Anxiety research in rodents has been frustrated by the small sample sizes typically used in experiments and the complexity of the mammalian brain,” says Adam Claridge-Chang of Duke-NUS Medical School in Singapore. “Many scientists in the rodent anxiety field would agree that this drug development failure does not imply a ‘failure to translate,’ but rather an inadequate understanding of the basic neurogenetic mechanisms of anxiety. “Using flies lends the advantages of a smaller animal brain, more sophisticated genetic tools, and greatly larger sample sizes. Evolutionary conservation of function between animal

In [ ]:
len(list(output[0]))

224

In [64]:
enc = tiktoken.get_encoding("gpt2")
enc.decode([config['end_token']])

'<|endoftext|>'